In [1]:
x = "stopit"

In [2]:
x[:-1]

'stopi'

In [239]:
import random
doc = "12345678901234567890123456789"
b = 10
l = random.randint(4, int(b*7/8))
print(f"b={b}\nl={l}")
doc = doc[0:l]
print(f"doc={doc}")

b=10
l=7
doc=1234567


In [266]:
lm = random.randint(1,int(l/2))
print(f"lm={lm}")
start = random.randint(2,l-lm-1)
print(f"start={start}")
prefix = doc[0:start-1]
print(f"prefix={prefix}")
masked_content = doc[start-1:start+lm-1]
print(f"masked_content={masked_content}")
suffix = doc[start+lm-1:]
print(f"suffix={suffix}\n")

lm=1
start=2
prefix=1
masked_content=2
suffix=34567


In [271]:
#rearrange: [prefix] MASK_CHAR [suffix] MASK_CHAR [masked_content] [pads]

MASK_CHAR = "\u2047" # the doublequestionmark character, for mask
PAD_CHAR = "\u25A1" # the empty square character, for pad
masked_string = prefix + MASK_CHAR + suffix + MASK_CHAR + masked_content
print(f"masked_string={masked_string}")
lp = b - len(masked_string) + 1
print(f"lp={lp}")
masked_string = masked_string + PAD_CHAR*lp
print(f"masked_string={masked_string}")
lr = len(masked_string)
print(f"lr={lr}")

masked_string=1⁇34567⁇2
lp=2
masked_string=1⁇34567⁇2□□
lr=11


In [273]:
x = masked_string[:-1]
print(f"x={x}")
y = masked_string[1:]
print(f"y={y}")

x=1⁇34567⁇2□
y=⁇34567⁇2□□


In [ ]:
import torch
x = torch.tensor([stoi[c] for c in x], dtype=torch.long)
y = torch.tensor([stoi[c] for c in y], dtype=torch.long)

In [283]:
import math
import torch
def precompute_rotary_emb(dim, max_positions):
    """
    RoPE uses the following sinusoidal functions to encode positions:

    cos(t theta_i) and sin(t theta_i)
        where t is the position and
              theta_i = 1/10000^(-2(i-1)/dim) for i in [1, dim/2]

    Since the maximum length of sequences is known, we can precompute
    these values to speed up training.

    Implement the precompute_rotary_emb function that returns a tensor of
    shape (max_positions, dim/2, 2) where the last dimension contains
    the cos and sin values for each position and each dimension of
    the embedding.
    """

    rope_cache = None
    # TODO: [part g]
    ### YOUR CODE HERE ###
    # pass

    def theta(i):
        return 1/10000**(-2*(i-1)/dim)

    assert dim % 2 == 0
    rope_cache = torch.zeros(max_positions, int(dim/2), 2)
    for t in range(max_positions):
        for i in range(int(dim/2)):
            rope_cache[t, i, 0] = math.cos((t+1) * theta(i+1))
            rope_cache[t, i, 1] = math.sin((t+1) * theta(i+1))
    ### END YOUR CODE ###
    return rope_cache

In [284]:
precompute_rotary_emb(6, 2)

tensor([[[ 0.5403,  0.8415],
         [-0.9018,  0.4321],
         [ 0.6990, -0.7151]],

        [[-0.4161,  0.9093],
         [ 0.6266, -0.7793],
         [-0.0229, -0.9997]]])

In [301]:
rope_cache = precompute_rotary_emb(6, 2 )
rope_cache

tensor([[[ 0.5403,  0.8415],
         [-0.9018,  0.4321],
         [ 0.6990, -0.7151]],

        [[-0.4161,  0.9093],
         [ 0.6266, -0.7793],
         [-0.0229, -0.9997]]])

In [302]:
    B = 1
    T = 2
    d = 6
    x = torch.rand(B, T, d)
    print(f'x = {x}')
    x = x.view(B, T, int(d/2), 2)
    print(f'x = {x}')
    print(f'x viewed as complex = {torch.view_as_complex(x)}')

x = tensor([[[0.3866, 0.7070, 0.0736, 0.6257, 0.7537, 0.2405],
         [0.0553, 0.1678, 0.4462, 0.5760, 0.2281, 0.4143]]])
x = tensor([[[[0.3866, 0.7070],
          [0.0736, 0.6257],
          [0.7537, 0.2405]],

         [[0.0553, 0.1678],
          [0.4462, 0.5760],
          [0.2281, 0.4143]]]])
x viewed as complex = tensor([[[0.3866+0.7070j, 0.0736+0.6257j, 0.7537+0.2405j],
         [0.0553+0.1678j, 0.4462+0.5760j, 0.2281+0.4143j]]])


In [303]:
def apply_rotary_emb(x, rope_cache):
    """Apply the RoPE to the input tensor x."""
    # TODO: [part g]
    # You might find the following functions useful to convert
    # between real and complex numbers:

    # torch.view_as_real - https://pytorch.org/docs/stable/generated/torch.view_as_real.html
    # torch.view_as_complex - https://pytorch.org/docs/stable/generated/torch.view_as_complex.html

    # Note that during inference, the length of the sequence might be different
    # from the length of the precomputed values. In this case, you should use
    # truncate the precomputed values to match the length of the sequence.

    rotated_x = None
    ### YOUR CODE HERE ###
    # pass
    print(f'RoTATING: {x}')
    print(f'shape(x) = {x.size()}')
    print(f'shape(rope_cache) = {rope_cache.size()}')
    B, T, d = x.size()
    assert d % 2 == 0
    # if d < len(rope_cache):
    #     rope_cache = rope_cache[:, :, :len(x)]
    complex_rope_cache = torch.view_as_complex(rope_cache)  # (T, d/2)
    complex_x = torch.view_as_complex(x.view(B, T, int(d/2), 2))  # (B, T, d/2)
    complex_rotated_x = torch.rand_like(complex_x)
    for b in range(B):
        for t in range(T):
            complex_rotated_x[b,t] = complex_rope_cache[t] * complex_x[b,t]
            
    rotated_x = torch.view_as_real(complex_rotated_x).view(B, T, d)
    
    ### END YOUR CODE ###
    return rotated_x

In [306]:
x = torch.rand(B, T, d)
print(f'x = {x}')
rotated_x = apply_rotary_emb(x, rope_cache)
print(f'rotated_x = {rotated_x}')

x = tensor([[[0.6121, 0.2404, 0.5566, 0.3531, 0.8683, 0.1742],
         [0.8408, 0.6745, 0.8937, 0.8837, 0.1527, 0.5294]]])
RoTATING: tensor([[[0.6121, 0.2404, 0.5566, 0.3531, 0.8683, 0.1742],
         [0.8408, 0.6745, 0.8937, 0.8837, 0.1527, 0.5294]]])
shape(x) = torch.Size([1, 2, 6])
shape(rope_cache) = torch.Size([2, 3, 2])
rotated_x = tensor([[[ 0.1284,  0.6450, -0.6545, -0.0780,  0.7315, -0.4991],
         [-0.9632,  0.4838,  1.2487, -0.1427,  0.5258, -0.1648]]])
